# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/le soir/sen.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [5]:
bigram_phrases = Phrases(sentences)

FileNotFoundError: [Errno 2] No such file or directory: '../data/le soir/sen.txt'

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [9]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [10]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [11]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [12]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [14]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [15]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [16]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [17]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [18]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=3, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 28min 15s
Wall time: 30min 25s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [19]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [20]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [21]:
model.wv["ministre"]

array([ -0.8592385 ,  -2.215879  ,  -0.7634464 ,   0.06558076,
        -4.440645  ,   2.677403  ,   4.0470934 ,  -1.8468516 ,
        -9.397952  ,  -3.7101028 ,  -4.815784  ,  -0.7650319 ,
        -3.8986797 ,   6.7658744 ,   3.0545046 ,   0.24993262,
         2.7208877 ,  -2.6435652 ,   4.636017  ,  -7.4115434 ,
         2.1912825 ,  -3.609059  ,   3.2000883 ,   1.368436  ,
        -2.030596  ,  -3.288343  ,  -6.870365  ,  -5.4437413 ,
         0.78009707,  -0.7077071 , -12.576853  ,  -8.8808155 ],
      dtype=float32)

### Calculer la similarité entre deux termes

In [22]:
model.wv.similarity("maison", "hebergement")

0.16111952

In [23]:
model.wv.similarity("agglomeration", "ville")

0.6291579

### Chercher les mots les plus proches d'un terme donné

In [24]:
model.wv.most_similar("saison", topn=10)

[('prochaine_saison', 0.8418573141098022),
 ('excursion', 0.8159013390541077),
 ('semaine', 0.8141476511955261),
 ('saison_estivale', 0.8134052753448486),
 ('promenade', 0.8044359087944031),
 ('premiere_moitie', 0.8029248714447021),
 ('competition', 0.7978430986404419),
 ('pentecote', 0.7944634556770325),
 ('foire', 0.7905126810073853),
 ('derniere_epreuve', 0.787421464920044)]

### Faire des recherches complexes à travers l'espace vectoriel

In [25]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('berlin', 0.8126980066299438), ('teheran', 0.785207986831665), ('new_york', 0.7729134559631348), ('budapest', 0.7685685753822327), ('shanghai', 0.7648318409919739), ('tokio', 0.763954222202301), ('tanger', 0.762924075126648), ('rome', 0.7595675587654114), ('prague', 0.7589729428291321), ('vienne', 0.7570575475692749)]
